## Introduksjon til klassifisering: Rens, forbered og visualiser dataene dine

I disse fire leksjonene vil du utforske et grunnleggende fokus innen klassisk maskinlæring - *klassifisering*. Vi skal gå gjennom bruken av ulike klassifiseringsalgoritmer med et datasett om alle de fantastiske kjøkkenene i Asia og India. Håper du er sulten!

<p >
   <img src="../../images/pinch.png"
   width="600"/>
   <figcaption>Feir pan-asiatiske kjøkken i disse leksjonene! Bilde av Jen Looper</figcaption>


<!--![Feir pan-asiatiske kjøkken i disse leksjonene! Bilde av Jen Looper](../../../../../../4-Classification/1-Introduction/solution/R/images/pinch.png)-->

Klassifisering er en form for [supervised learning](https://wikipedia.org/wiki/Supervised_learning) som har mye til felles med regresjonsteknikker. I klassifisering trener du en modell til å forutsi hvilken `kategori` et element tilhører. Hvis maskinlæring handler om å forutsi verdier eller navn på ting ved hjelp av datasett, faller klassifisering generelt inn i to grupper: *binær klassifisering* og *multiklassifisering*.

Husk:

-   **Lineær regresjon** hjalp deg med å forutsi forholdet mellom variabler og lage nøyaktige spådommer om hvor et nytt datapunkt ville falle i forhold til den linjen. For eksempel kunne du forutsi numeriske verdier som *hva prisen på et gresskar ville være i september vs. desember*.

-   **Logistisk regresjon** hjalp deg med å oppdage "binære kategorier": ved dette prisnivået, *er dette gresskaret oransje eller ikke-oransje*?

Klassifisering bruker ulike algoritmer for å bestemme andre måter å avgjøre en datapunkts etikett eller klasse. La oss jobbe med dette kjøkkendatasettet for å se om vi, ved å observere en gruppe ingredienser, kan bestemme opprinnelsen til kjøkkenet.

### [**Quiz før leksjonen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/19/)

### **Introduksjon**

Klassifisering er en av de grunnleggende aktivitetene for maskinlæringsforskere og dataforskere. Fra enkel klassifisering av en binær verdi ("er denne e-posten spam eller ikke?") til kompleks bildeklassifisering og segmentering ved hjelp av datamaskinsyn, er det alltid nyttig å kunne sortere data i klasser og stille spørsmål om dem.

For å formulere prosessen på en mer vitenskapelig måte, skaper klassifiseringsmetoden din en prediktiv modell som gjør det mulig å kartlegge forholdet mellom inputvariabler og outputvariabler.

<p >
   <img src="../../images/binary-multiclass.png"
   width="600"/>
   <figcaption>Binære vs. multiklasseproblemer som klassifiseringsalgoritmer kan håndtere. Infografikk av Jen Looper</figcaption>



Før vi starter prosessen med å rense dataene våre, visualisere dem og forberede dem for våre ML-oppgaver, la oss lære litt om de ulike måtene maskinlæring kan brukes til å klassifisere data.

Avledet fra [statistikk](https://wikipedia.org/wiki/Statistical_classification), bruker klassifisering med klassisk maskinlæring funksjoner som `røyker`, `vekt` og `alder` for å bestemme *sannsynligheten for å utvikle X sykdom*. Som en supervised learning-teknikk, lik de regresjonsøvelsene du utførte tidligere, er dataene dine merket, og ML-algoritmene bruker disse etikettene til å klassifisere og forutsi klasser (eller 'funksjoner') i et datasett og tilordne dem til en gruppe eller et utfall.

✅ Ta et øyeblikk til å forestille deg et datasett om kjøkken. Hva ville en multiklassemodell kunne svare på? Hva ville en binær modell kunne svare på? Hva om du ønsket å avgjøre om et gitt kjøkken sannsynligvis bruker bukkehornkløver? Hva om du ville se om du, gitt en pose med stjerneanis, artisjokker, blomkål og pepperrot, kunne lage en typisk indisk rett?

### **Hei 'klassifikator'**

Spørsmålet vi ønsker å stille om dette kjøkkendatasettet er faktisk et **multiklasse-spørsmål**, ettersom vi har flere potensielle nasjonale kjøkken å jobbe med. Gitt en gruppe ingredienser, hvilken av disse mange klassene passer dataene inn i?

Tidymodels tilbyr flere ulike algoritmer for å klassifisere data, avhengig av hvilken type problem du ønsker å løse. I de neste to leksjonene vil du lære om flere av disse algoritmene.

#### **Forutsetning**

For denne leksjonen trenger vi følgende pakker for å rense, forberede og visualisere dataene våre:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling av R-pakker](https://www.tidyverse.org/packages) designet for å gjøre dataanalyse raskere, enklere og morsommere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) rammeverket er en [samling av pakker](https://www.tidymodels.org/packages/) for modellering og maskinlæring.

-   `DataExplorer`: [DataExplorer-pakken](https://cran.r-project.org/web/packages/DataExplorer/vignettes/dataexplorer-intro.html) er ment å forenkle og automatisere EDA-prosessen og rapportgenerering.

-   `themis`: [themis-pakken](https://themis.tidymodels.org/) gir ekstra oppskritt for å håndtere ubalanserte data.

Du kan installere dem som:

`install.packages(c("tidyverse", "tidymodels", "DataExplorer", "here"))`

Alternativt sjekker skriptet nedenfor om du har de nødvendige pakkene for å fullføre dette modulen og installerer dem for deg hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, DataExplorer, themis, here)

Vi vil senere laste inn disse fantastiske pakkene og gjøre dem tilgjengelige i vår nåværende R-økt. (Dette er kun for illustrasjon, `pacman::p_load()` har allerede gjort det for deg)


## Øvelse - rengjør og balanser dataene dine

Den første oppgaven før du starter dette prosjektet, er å rengjøre og **balansere** dataene dine for å oppnå bedre resultater.

La oss bli kjent med dataene! 🕵️


In [ ]:
# Import data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# View the first 5 rows
df %>% 
  slice_head(n = 5)


Interessant! Ut fra utseendet ser det ut til at den første kolonnen er en slags `id`-kolonne. La oss få litt mer informasjon om dataene.


In [ ]:
# Basic information about the data
df %>%
  introduce()

# Visualize basic information above
df %>% 
  plot_intro(ggtheme = theme_light())

Fra resultatet kan vi umiddelbart se at vi har `2448` rader og `385` kolonner og `0` manglende verdier. Vi har også 1 diskret kolonne, *cuisine*.

## Øvelse - lære om matretter

Nå begynner arbeidet å bli mer interessant. La oss utforske fordelingen av data, per matrett.


In [ ]:
# Count observations per cuisine
df %>% 
  count(cuisine) %>% 
  arrange(n)

# Plot the distribution
theme_set(theme_light())
df %>% 
  count(cuisine) %>% 
  ggplot(mapping = aes(x = n, y = reorder(cuisine, -n))) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("cuisine")

Det finnes et begrenset antall kjøkken, men fordelingen av data er ujevn. Det kan du fikse! Før du gjør det, utforsk litt mer.

La oss deretter tilordne hvert kjøkken til sin egen tibble og finne ut hvor mye data som er tilgjengelig (rader, kolonner) per kjøkken.

> En [tibble](https://tibble.tidyverse.org/) er en moderne data frame.

<p >
   <img src="../../images/dplyr_filter.jpg"
   width="600"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>


In [ ]:
# Create individual tibble for the cuisines
thai_df <- df %>% 
  filter(cuisine == "thai")
japanese_df <- df %>% 
  filter(cuisine == "japanese")
chinese_df <- df %>% 
  filter(cuisine == "chinese")
indian_df <- df %>% 
  filter(cuisine == "indian")
korean_df <- df %>% 
  filter(cuisine == "korean")


# Find out how much data is available per cuisine
cat(" thai df:", dim(thai_df), "\n",
    "japanese df:", dim(japanese_df), "\n",
    "chinese_df:", dim(chinese_df), "\n",
    "indian_df:", dim(indian_df), "\n",
    "korean_df:", dim(korean_df))

## **Øvelse - Oppdage toppingredienser per kjøkken ved hjelp av dplyr**

Nå kan du dykke dypere inn i dataene og finne ut hvilke ingredienser som er typiske for hvert kjøkken. Du bør fjerne gjentakende data som skaper forvirring mellom kjøkken, så la oss lære mer om dette problemet.

Lag en funksjon `create_ingredient()` i R som returnerer et ingrediens-datasett. Denne funksjonen vil starte med å fjerne en unyttig kolonne og sortere ingrediensene etter antall.

Den grunnleggende strukturen til en funksjon i R er:

`myFunction <- function(arglist){`

**`...`**

**`return`**`(value)`

`}`

En ryddig introduksjon til R-funksjoner finner du [her](https://skirmer.github.io/presentations/functions_with_r.html#1).

La oss komme i gang! Vi skal bruke [dplyr-verb](https://dplyr.tidyverse.org/) som vi har lært i tidligere leksjoner. Som en oppsummering:

-   `dplyr::select()`: hjelper deg med å velge hvilke **kolonner** du vil beholde eller ekskludere.

-   `dplyr::pivot_longer()`: hjelper deg med å "forlenge" data, øke antall rader og redusere antall kolonner.

-   `dplyr::group_by()` og `dplyr::summarise()`: hjelper deg med å finne sammendragsstatistikk for ulike grupper og sette dem i en oversiktlig tabell.

-   `dplyr::filter()`: lager et datasett som kun inneholder rader som oppfyller dine betingelser.

-   `dplyr::mutate()`: hjelper deg med å lage eller endre kolonner.

Sjekk ut denne [*kunst*-fylte learnr-veiledningen](https://allisonhorst.shinyapps.io/dplyr-learnr/#section-welcome) av Allison Horst, som introduserer noen nyttige datahåndteringsfunksjoner i dplyr *(en del av Tidyverse)*.


In [ ]:
# Creates a functions that returns the top ingredients by class

create_ingredient <- function(df){
  
  # Drop the id column which is the first colum
  ingredient_df = df %>% select(-1) %>% 
  # Transpose data to a long format
    pivot_longer(!cuisine, names_to = "ingredients", values_to = "count") %>% 
  # Find the top most ingredients for a particular cuisine
    group_by(ingredients) %>% 
    summarise(n_instances = sum(count)) %>% 
    filter(n_instances != 0) %>% 
  # Arrange by descending order
    arrange(desc(n_instances)) %>% 
    mutate(ingredients = factor(ingredients) %>% fct_inorder())
  
  
  return(ingredient_df)
} # End of function

Nå kan vi bruke funksjonen for å få en idé om de ti mest populære ingrediensene etter kjøkken. La oss teste den med `thai_df`.


In [ ]:
# Call create_ingredient and display popular ingredients
thai_ingredient_df <- create_ingredient(df = thai_df)

thai_ingredient_df %>% 
  slice_head(n = 10)

I forrige seksjon brukte vi `geom_col()`, la oss se hvordan du også kan bruke `geom_bar` til å lage stolpediagrammer. Bruk `?geom_bar` for videre lesing.


In [ ]:
# Make a bar chart for popular thai cuisines
thai_ingredient_df %>% 
  slice_head(n = 10) %>% 
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "steelblue") +
  xlab("") + ylab("")

In [ ]:
# Get popular ingredients for Japanese cuisines and make bar chart
create_ingredient(df = japanese_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "darkorange", alpha = 0.8) +
  xlab("") + ylab("")


Hva med de kinesiske kjøkkenene?


In [ ]:
# Get popular ingredients for Chinese cuisines and make bar chart
create_ingredient(df = chinese_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "cyan4", alpha = 0.8) +
  xlab("") + ylab("")

In [ ]:
# Get popular ingredients for Indian cuisines and make bar chart
create_ingredient(df = indian_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "#041E42FF", alpha = 0.8) +
  xlab("") + ylab("")

In [ ]:
# Get popular ingredients for Korean cuisines and make bar chart
create_ingredient(df = korean_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "#852419FF", alpha = 0.8) +
  xlab("") + ylab("")

Fra datavisualiseringene kan vi nå fjerne de vanligste ingrediensene som skaper forvirring mellom ulike kjøkken, ved å bruke `dplyr::select()`.

Alle elsker ris, hvitløk og ingefær!


In [ ]:
# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger))

# Display new data set
df_select %>% 
  slice_head(n = 5)

## Forhåndsbehandling av data med oppskrifter 👩‍🍳👨‍🍳 - Håndtering av ubalanserte data ⚖️

<p >
   <img src="../../images/recipes.png"
   width="600"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>

Siden denne leksjonen handler om matretter, må vi sette `recipes` i kontekst.

Tidymodels tilbyr nok en praktisk pakke: `recipes` - en pakke for forhåndsbehandling av data.


La oss se på fordelingen av våre kjøkken igjen.


In [ ]:
# Distribution of cuisines
old_label_count <- df_select %>% 
  count(cuisine) %>% 
  arrange(desc(n))

old_label_count

Som du kan se, er det en ganske ujevn fordeling i antall kjøkken. Koreanske kjøkken er nesten tre ganger så mange som thailandske kjøkken. Ubalanserte data har ofte negative effekter på modellens ytelse. Tenk på en binær klassifisering. Hvis mesteparten av dataene dine tilhører én klasse, vil en ML-modell ofte forutsi denne klassen oftere, bare fordi det finnes mer data for den. Å balansere dataene tar skjevheten i datasettet og bidrar til å fjerne denne ubalansen. Mange modeller presterer best når antall observasjoner er likt, og har derfor en tendens til å slite med ubalanserte data.

Det finnes hovedsakelig to måter å håndtere ubalanserte datasett på:

-   legge til observasjoner i minoritetsklassen: `Over-sampling`, f.eks. ved bruk av en SMOTE-algoritme

-   fjerne observasjoner fra majoritetsklassen: `Under-sampling`

La oss nå demonstrere hvordan man kan håndtere ubalanserte datasett ved hjelp av en `recipe`. En recipe kan sees på som en oppskrift som beskriver hvilke steg som skal brukes på et datasett for å gjøre det klart for dataanalyse.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing data
cuisines_recipe <- recipe(cuisine ~ ., data = df_select) %>% 
  step_smote(cuisine)

cuisines_recipe

La oss bryte ned våre forhåndsprosesseringssteg.

-   Kallet til `recipe()` med en formel forteller oppskriften *rollene* til variablene ved å bruke `df_select`-data som referanse. For eksempel har `cuisine`-kolonnen blitt tildelt rollen som `outcome`, mens de andre kolonnene har blitt tildelt rollen som `predictor`.

-   [`step_smote(cuisine)`](https://themis.tidymodels.org/reference/step_smote.html) lager en *spesifikasjon* av et oppskriftssteg som syntetisk genererer nye eksempler av minoritetsklassen ved hjelp av nærmeste naboer til disse tilfellene.

Nå, hvis vi ønsket å se de forhåndsprosesserte dataene, måtte vi [**`prep()`**](https://recipes.tidymodels.org/reference/prep.html) og [**`bake()`**](https://recipes.tidymodels.org/reference/bake.html) oppskriften vår.

`prep()`: estimerer de nødvendige parameterne fra et treningssett som senere kan brukes på andre datasett.

`bake()`: tar en ferdigbehandlet oppskrift og anvender operasjonene på et hvilket som helst datasett.


In [ ]:
# Prep and bake the recipe
preprocessed_df <- cuisines_recipe %>% 
  prep() %>% 
  bake(new_data = NULL) %>% 
  relocate(cuisine)

# Display data
preprocessed_df %>% 
  slice_head(n = 5)

# Quick summary stats
preprocessed_df %>% 
  introduce()

La oss nå sjekke fordelingen av våre kjøkken og sammenligne dem med de ubalanserte dataene.


In [ ]:
# Distribution of cuisines
new_label_count <- preprocessed_df %>% 
  count(cuisine) %>% 
  arrange(desc(n))

list(new_label_count = new_label_count,
     old_label_count = old_label_count)

Nam! Dataen er fin og ren, balansert, og veldig deilig 😋!

> Vanligvis brukes en oppskrift som en forprosessering for modellering, hvor den definerer hvilke steg som skal brukes på et datasett for å gjøre det klart for modellering. I så fall brukes vanligvis en `workflow()` (som vi allerede har sett i tidligere leksjoner) i stedet for å manuelt estimere en oppskrift.
>
> Derfor trenger du vanligvis ikke å **`prep()`** og **`bake()`** oppskrifter når du bruker tidymodels, men de er nyttige funksjoner å ha i verktøykassen for å bekrefte at oppskriftene gjør det du forventer, som i vårt tilfelle.
>
> Når du **`bake()`** en forberedt oppskrift med **`new_data = NULL`**, får du tilbake dataene du ga da du definerte oppskriften, men med de forprosesseringstrinnene som er utført.

La oss nå lagre en kopi av disse dataene for bruk i fremtidige leksjoner:


In [ ]:
# Save preprocessed data
write_csv(preprocessed_df, "../../../data/cleaned_cuisines_R.csv")

Denne nye CSV-filen kan nå finnes i rotmappen for data.

**🚀Utfordring**

Denne læreplanen inneholder flere interessante datasett. Gå gjennom `data`-mappene og se om noen inneholder datasett som kan være egnet for binær eller flervalgsklassifisering. Hvilke spørsmål ville du stille til dette datasettet?

## [**Quiz etter forelesning**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/20/)

## **Gjennomgang og selvstudium**

-   Sjekk ut [pakken themis](https://github.com/tidymodels/themis). Hvilke andre teknikker kan vi bruke for å håndtere ubalanserte data?

-   Tidy models [referansenettsted](https://www.tidymodels.org/start/).

-   H. Wickham og G. Grolemund, [*R for Data Science: Visualize, Model, Transform, Tidy, and Import Data*](https://r4ds.had.co.nz/).

#### TAKK TIL:

[`Allison Horst`](https://twitter.com/allison_horst/) for å lage de fantastiske illustrasjonene som gjør R mer innbydende og engasjerende. Finn flere illustrasjoner i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) og [Jen Looper](https://www.twitter.com/jenlooper) for å ha laget den originale Python-versjonen av denne modulen ♥️

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="600"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
